In [ ]:
using Match
import Base: setindex!

# Virtual Machine 

## AST walking 

```julia

let (e::Environment)(name)
   if name in keys(e.names)
         return e.names[name]
      e.outer is not e
         return e.outer(e)
      else
         error("Error: $name not defined")
```

In [518]:
struct Environment
    names::Dict{Symbol, Atom}
    outer::Environment
    function Environment()
        e = new()
        e.names = Dict{Symbol, Atom}()
        e.outer = e
        return e
    end
    function Environment(outer)
        e = new()
        e.names = Dict{Symbol, Atom}()
        e.outer = outer
        return e
    end
    function Enviroment(outer, keys, values)
        e = new()
        e.names = Dict(zip(keys, values))
        e.outer = outer
        return e
    end
end

function (e::Environment)(name)
    if name in keys(e.names)
        return e.names[name]
    elseif !(e.outer === e)
        return e.outer(name)
    else
        error("Error: $name not defined")
    end
end

# function setindex!(e::Environment, value, name::Symbol)
#     if name in keys(e.names)
#         e.names[name] = value
#     elseif !(e.outer === e) 
#         e.outer[name] = value
#     else
#         error("Error: $name not defined")
#     end
#     return nothing
# end
function setindex!(e::Environment, value, name::Symbol)
    e.names[name] = value
    return nothing
end

setindex! (generic function with 91 methods)

In [203]:
struct Atom{T}
    val::T
end
function val(a::Atom)
    return a.val
end
struct SExpr
    expr::Tuple
end
struct Macro{T}
    func::T
end
function (m::Macro)(args...)
    return m.func(args...)
end
function atomtype(a::Atom{T}) where T
    return T
end
function se(args...)
    return SExpr((x -> x isa Atom || x isa SExpr ? x : Atom(x)).(args))
end
function at(arg)
    return Atom(arg)
end

at (generic function with 1 method)

In [500]:
BUILTINS = Dict(
    :+ => (+), 
    :* => (*),
    :list => (args...) -> SExpr(args),
    :lambda => :lambda,
    :macro  => :macro,
    :quote => :quote,
)
DEFAULT_ENV = name -> name in keys(BUILTINS) ? BUILTINS[name] : error("Error: $(name) not defined")

# function create_lambda(ast, env)
#     return (args...) -> ast_walk(
#             ast.expr[3],
#             env = begin
#                 nameargs = Dict(zip(val.(ast.expr[2].expr), args))
#                 name -> name in keys(nameargs) ? nameargs[name] : env(name) 
#             end
#         )
# end
function create_lambda(ast, env)
    return 

function ast_walk(ast::SExpr; env=DEFAULT_ENV)
    @match ast.expr[1] begin
        at(:lambda) => create_lambda(ast, env)
        at(:macro)  => Macro(create_lambda(ast, env))
        at(:quote)  => ast.expr[2]
        at(:begin)  => for a in ast.expr[2:end]; ast_walk(a, env=env) end
        at(:set!)   => (env[ast.expr[2].value] = ast_walk(ast.expr[3], env=env))
        _ => begin
            item = ast_walk(ast.expr[1], env=env)
            @match item begin
                _::Function => item(ast_walk.(ast.expr[2:end], env=env)...)
                _::Macro    => ast_walk(item(ast.expr[2:end]...), env=env)
                _ => error("Error: not callable")
            end
        end
    end
end
function ast_walk(ast::Atom; env=DEFAULT_ENV)
    return atomtype(ast) == Symbol ? env(ast.val) : ast.val
end

ast_walk (generic function with 3 methods)

In [501]:
ast_walk(se(:+, se(:*, 3, 4), se(:*, 3, 3)))

21

In [502]:
ast_walk(se(se(:lambda, se(:x,), se(:+, se(:*, :x, 2), 1)), 3))

7

In [503]:
ast_walk(se(se(:macro, se(:x), se(:list, se(:quote, :+), :x, :x)), 4))

8

In [504]:
ast_walk(se(:lambda, se(:let), se(:let, :x, 3, se(:+, :x, 4))))

#825 (generic function with 1 method)

In [505]:
ast_walk(se(:macro, se(:name, :value, :body), se(:list, se(:list, :lambda, se(:name), :body),  :value)))

Macro{var"#825#827"{SExpr, var"#823#824"}}(var"#825#827"{SExpr, var"#823#824"}(SExpr((Atom{Symbol}(:macro), SExpr((Atom{Symbol}(:name), Atom{Symbol}(:value), Atom{Symbol}(:body))), SExpr((Atom{Symbol}(:list), SExpr((Atom{Symbol}(:list), Atom{Symbol}(:lambda), SExpr((Atom{Symbol}(:name),)), Atom{Symbol}(:body))), Atom{Symbol}(:value))))), var"#823#824"()))

In [506]:
ast_walk(se(:quote, se(:x, :y)))

SExpr((Atom{Symbol}(:x), Atom{Symbol}(:y)))

In [507]:
ast_walk(se(:quote, se(:lambda, se(:x), se(:+, :x, 3))))

SExpr((Atom{Symbol}(:lambda), SExpr((Atom{Symbol}(:x),)), SExpr((Atom{Symbol}(:+), Atom{Symbol}(:x), Atom{Int64}(3)))))

In [514]:
ast_walk(
    se(se(:lambda, se(:let), se(:let, :x, 3, se(:*, se(:+, :x, :x), 4))),
       se(:macro, se(:name, :value, :body), se(:list, se(:list, se(:quote, :lambda), se(:list, :name), :body),  :value)))
)

24

In [515]:
se(se(:lambda, se(:let), se(:let, :x, 3, se(:+, :x, 4))),
       se(:macro, se(:name, :value, :body), se(:list, se(:list, se(:quote, :lambda), se(:list, :name), :body),  :value)))

SExpr((SExpr((Atom{Symbol}(:lambda), SExpr((Atom{Symbol}(:let),)), SExpr((Atom{Symbol}(:let), Atom{Symbol}(:x), Atom{Int64}(3), SExpr((Atom{Symbol}(:+), Atom{Symbol}(:x), Atom{Int64}(4))))))), SExpr((Atom{Symbol}(:macro), SExpr((Atom{Symbol}(:name), Atom{Symbol}(:value), Atom{Symbol}(:body))), SExpr((Atom{Symbol}(:list), SExpr((Atom{Symbol}(:list), SExpr((Atom{Symbol}(:quote), Atom{Symbol}(:lambda))), SExpr((Atom{Symbol}(:list), Atom{Symbol}(:name))), Atom{Symbol}(:body))), Atom{Symbol}(:value)))))))

```
((lambda (let)

    (let x (+ 4 5)
    (let y (+ x 4)
           (+ x y)))

)(macro (name value body) (list (list 'lambda) (list name) body)))
```

In [516]:
ast_walk(
    se(se(:lambda, se(:let),
            
            se(:let, :x, se(:+,  4, 5),
            se(:let, :y, se(:+, :x, 4), 
                         se(:+, :x, :y)))
            
    ), se(:macro, se(:name, :value, :body), se(:list, se(:list, se(:quote, :lambda), se(:list, :name), :body),  :value)))
)

22

## Simple Stack Machine

Translation as code execution.
